In [1]:
import os
import subprocess
import numpy as np
import cv2
import shutil

In [2]:
dirlist = subprocess.check_output(["find", "/run/user/1000/gvfs/dav:host=nextcloud.kamaro-engineering.de,ssl=true,user=jbarthel,prefix=%2Fremote.php%2Fdav/files/jbarthel/Datenverarbeitung/Datensätze/Löwenzahn/Plastik Modell/mit"])

In [3]:
images = [e for e in dirlist.decode('utf-8').splitlines() if e[-3:].lower() in ["jpg","png"]]

In [4]:
names = [f.lower().replace(":","_").replace(" ","_").replace("/markiert","").split("/")[-1] for f in images]

In [5]:
filenames_labels = dict()
filenames_images = dict()

for name, image in zip(names, images):
    if name.endswith("jpg"):
        filenames_images[name[:-4]] = image
    elif name.endswith("png"):
        filenames_labels[name[:-4]] = image
    else:
        raise ValueError("Invalid filename "+name+" for image "+image)

In [6]:
keys = list(filenames_labels.keys())

In [9]:
MARKER_DANDELION = np.array([255,255,0])
MARKER_ALCOHOLIC = np.array([0,255,255])

def load_label(name):
    raw = cv2.imread(filenames_labels[name])[:,:,::-1]
    mask_dandelion = (np.abs(raw - MARKER_DANDELION).mean(axis=-1) < 20 ).astype(np.uint8)
    mask_alcoholic = (np.abs(raw - MARKER_ALCOHOLIC).mean(axis=-1) < 20 ).astype(np.uint8)
    mask_none = np.ones_like(mask_dandelion) - mask_dandelion - mask_alcoholic
    return np.stack([mask_dandelion,mask_alcoholic, mask_none], axis=-1) * 255

In [10]:
for key in keys:
    out_image = key + ".jpg"
    out_label = key + ".png"
    
    if not os.path.isfile(out_image):
        print("copying image", out_image)
        shutil.copyfile(filenames_images[key], out_image)
    else:
        print("skipping image ", out_image)
        
    if not os.path.isfile(out_label):
        print("copying label", out_label)
        label = load_label(key)
        cv2.imwrite(out_label, label)
    else:
        print("skipping label", out_label)

skipping image  img_2022-05-31t18_33_06.425536.jpg
copying label img_2022-05-31t18_33_06.425536.png
skipping image  img_2022-05-31t18_33_21.992278.jpg
copying label img_2022-05-31t18_33_21.992278.png
skipping image  img_2022-05-31t18_33_25.827106.jpg
copying label img_2022-05-31t18_33_25.827106.png
skipping image  img_2022-05-31t18_33_40.293286.jpg
copying label img_2022-05-31t18_33_40.293286.png
skipping image  img_2022-05-31t18_33_43.294326.jpg
copying label img_2022-05-31t18_33_43.294326.png
skipping image  img_2022-05-31t18_33_59.593450.jpg
copying label img_2022-05-31t18_33_59.593450.png
skipping image  img_2022-05-31t18_34_06.360986.jpg
copying label img_2022-05-31t18_34_06.360986.png
skipping image  img_2022-05-31t18_34_19.526196.jpg
copying label img_2022-05-31t18_34_19.526196.png
skipping image  img_2022-05-31t18_34_21.061677.jpg
copying label img_2022-05-31t18_34_21.061677.png
skipping image  img_2022-05-31t18_43_12.295065.jpg
copying label img_2022-05-31t18_43_12.295065.png
